# Analyze Data

In [ ]:
devtools::install_version("ggpubr", version = "0.5.0", upgrade = "never", dependencies = "Imports")
#if (!requireNamespace("tidyr")) {
#    install.packages("tidyr")
# install.packages("ggpubr")
#}
#devtools::install.version("tidyr", version = "1.3.0")

In [ ]:
if (!requireNamespace("ggfortify")) {
    install.packages("ggfortify")
}
if(!requireNamespace("labelled")) {
  install.packages("labelled")
}
if(!requireNamespace("gtsummary")) {
  install.packages("gtsummary")
}

In [ ]:
library(tidyr)
library(arrow, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(ggplot2)
library(janitor)
library(broom)
library(emmeans)
library(scales)
library(survminer)
library(survival)
library(ggfortify)
library(gtools)
library(MASS, warn.conflicts = FALSE)
library(xtable)
library(bit64, warn.conflicts = FALSE)
library(table1)
library(patchwork)
library(labelled)
library(gtsummary)
library(gt)

In [ ]:
source(here::here("R", "theme_truveta.r"))

In [ ]:
initialize_theme_truveta()

In [ ]:
results_dir <- here::here("results")
data_dir <- here::here("data")
dir.create(data_dir, recursive = TRUE, showWarnings = FALSE)

In [ ]:
write_as_xtable <- function(table1, filepath, type = 'latex', coerce_to_df = TRUE, ...) {
  if(coerce_to_df) {
    table1 <- as.data.frame(table1)
  } 

  rownames(table1) <- NULL

  xtable::xtable(table1, ...) |>
    xtable::print.xtable(
      type = type, 
      file = filepath,
      include.rownames = FALSE,
      comment = FALSE,
      table.placement = '!htbp',
      format.args = list(big.mark = ',')
    )

  return(filepath)
}

## Load + Prepare Data

In [ ]:
df <- arrow::read_parquet(file.path(data_dir, "feature_table.parquet"))
#df |> head(5)

In [ ]:
nrow(df)
nrow(df[!is.na(df$PersonId),])
sum(is.na(df$PersonId))

In [ ]:
# Remove invalid last encounter start date time (assume these people had no encounters after T0)
sum(is.na(df$LastEncounterStartDateTime))

# df <- 
#   df |>
#   filter(!is.na(LastEncounterStartDateTime) & Exclude_censoring != 1)

In [ ]:
#Make stuff pretty
# Factor categorical variables
df <- df |> 
  mutate(
    across(
      c(contains(c('conditions','disability','prexist_cardiac_exclusion','chronic_diabetes_conditions'))), 
      ~ factor(.x, levels = c(0, 1), labels = c('No', 'Yes'))
    ),
    # Replace Black or African American with Black*
    Race = dplyr::if_else(as.character(Race) %in% "Black or African American", "Black", as.character(Race))
  )
black_footnote <- "*Black refers to Black or African American Individuals"


## Regroup age groups to be more clinically relevant for pregnancy
df <- df |>
  mutate(
    AgeGroup = 
      dplyr::case_when(
        AgeInYears < 18 ~ "0-17",
        AgeInYears >=18 & AgeInYears < 25 ~ "18-24",
        AgeInYears >= 25 & AgeInYears < 35 ~ "25-34",
        AgeInYears >= 35 & AgeInYears < 45 ~ "35-44",
        AgeInYears >= 45 ~ "45+"
      ),
    AgeGroup = factor(AgeGroup, levels =c("0-17", "18-24", "25-34", "35-44","45+"))
  )

df <- 
  df |> 
  mutate(
    preeclampsia_eclampsia_pretty = 
      factor(
        Eclampsia_binary, 
        levels = c(0, 1), 
        labels = c("No Preeclampsia/Eclampsia", "Preeclampsia/Eclampsia")
      ),
    disabled2 = factor(disabled, levels = c(0,1), labels = c("No", "Yes")),
    Advanced = factor(Advanced, levels = c(0,1), labels = c("No", "Yes")),
    Teen = factor(Teen, levels = c(0,1), labels = c("No", "Yes")),
       CountHBP = factor(CountHBP, levels = c(0,1), labels = c("No", "Yes")),
      heart_failure_pretty =
          factor(
              event_HF,
              levels = c(0,1),
              labels = c("No Heart Failure", "Heart Failure")
              )
    
  )

In [ ]:
# Relevel, lump categories with small numbers (may eventually replace with a more thoughtful grouping, but this is fine for now)
df_surv <- 
  df |>
  mutate(
    race = forcats::fct_lump_prop(Race, .05),
    race = ifelse(race == "Other", "Other Race", as.character(race)),           
    ethnicity = forcats::fct_lump_prop(Ethnicity, .05),
    ethnicity = ifelse(ethnicity == "Other", "Not Hispanic or Latino", as.character(ethnicity)), 
    race = forcats::fct_relevel(race, c("Black", "Asian", "White", "Other Race")),
    ethnicity = forcats::fct_relevel(ethnicity, c("Hispanic or Latino","Not Hispanic or Latino"))
  )

table(df_surv$race)
table(df_surv$ethnicity)
table(df_surv$AgeGroup)

In [ ]:
# Create event and time indicators for survival analysis
df_surv <- 
  df_surv |> 
  mutate(
    across(c(contains('DateTime'), 'T0'),~lubridate::as_datetime(.x)),
    first_event_hf = pmin(ConditionDateTime_HF, LastEncounterStartDateTime, na.rm = TRUE),
    event_hf = tidyr::replace_na((first_event_hf == ConditionDateTime_HF)*1,0),
    time_hf = as.integer(difftime(first_event_hf, T0, units = "days")),
    time90_hf = ifelse(time_hf > 90, 90, time_hf),
    event90_hf = ifelse(time_hf > 90, 0, event_hf),

  )

df_surv <- 
  df_surv |> 
  mutate(
       event_hfx = factor(event_hf, levels = c(0,1), labels = c("No", "Yes")),
  )

# Table 1

In [ ]:
df_tab1 <- 
  df_surv |>
  dplyr::select(
    time_hf,
    event_hfx,
    preeclampsia_eclampsia_pretty,
    race,
    ethnicity,
    AgeGroup,
    bleeding_conditions,
    chronic_diabetes_conditions,
    gest_diabetes_conditions,
    CountHBP,
    prexist_cardiac_exclusion,
    bmiabove40_conditions,
    sud_conditions,
    disabled2,
    oci
  )

table1::label(df_tab1$time_hf) <- 'Time till Heart Failure'
table1::label(df_tab1$event_hfx) <- 'Heart Failure Event'
table1::label(df_tab1$preeclampsia_eclampsia_pretty) <- 'Preeclampsia/Eclampsia'
table1::label(df_tab1$race) <- 'Race'
table1::label(df_tab1$ethnicity) <- 'Ethnicity'
table1::label(df_tab1$AgeGroup) <- 'Age Group'
table1::label(df_tab1$bleeding_conditions) <- 'Bleeding Conditions'
table1::label(df_tab1$chronic_diabetes_conditions) <- 'Pregestational Diabetes'
table1::label(df_tab1$gest_diabetes_conditions) <- 'Gestational Diabetes'
table1::label(df_tab1$CountHBP) <- 'Pregestational Hypertension'
table1::label(df_tab1$prexist_cardiac_exclusion) <- 'Other Preexisting Cardiac Conditions'
table1::label(df_tab1$bmiabove40_conditions) <- 'BMI \U2265 40'
table1::label(df_tab1$sud_conditions) <- 'Substance Use Disorder'
table1::label(df_tab1$disabled2) <- 'Disability'
table1::label(df_tab1$oci) <- 'Obstetric Comorbidity Index'

tab1 <- 
  table1::table1(
    ~ event_hfx +
      race + 
      ethnicity + 
      AgeGroup + 
      bleeding_conditions +
      chronic_diabetes_conditions + 
      gest_diabetes_conditions + 
      CountHBP + 
      prexist_cardiac_exclusion +
      bmiabove40_conditions + 
      sud_conditions + 
      disabled2 + 
      oci | preeclampsia_eclampsia_pretty,
    data = df_tab1,
    big.mark = ','
  ) 


tab1_df <- as.data.frame(tab1, make.names = FALSE)

tab1_df

In [ ]:
add.to.row <- list()
add.to.row$pos <- list()
add.to.row$pos[[1]] <- c(0)
add.to.row$command <- 
  paste0(
    "\\hline \n",
    "\\endfirsthead \n",
    "\\multicolumn{4}{p{\\textwidth}}{{ \\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\ \n",
    "\\hline Feature & No Preeclampsia/Eclampsia & Preeclampsia/Eclampsia & Overall \\\\ \\hline \n",
    "\\endhead \n",
    "\\hline \\multicolumn{4}{p{\\textwidth}}{{Continued on next page}} \\\\ \\hline \n",
    "\\endfoot \n",
    "\\hline \n",
    "\\endlastfoot \n"
  )

#align <- c('', "p{0.2\\textwidth}", "p{0.2\\textwidth}", "p{0.6\\textwidth}")

caption <- 
  paste0(
    'Demographic characteristics of people that delivered at or after January 1, 2018 and their comorbidities. '
  )

label <- 'tab:table_1'

xtable::print.xtable(
  xtable::xtable(
    tab1_df,
    label = label,
    caption = caption,
    floating = FALSE#,
    #align = align
  ),
  type = 'latex', 
  file = file.path(results_dir, "table_1.tex"), 
  include.rownames = FALSE,
  comment = FALSE,
  timestamp = NULL,
  floating = FALSE,
  add.to.row = add.to.row,
  tabular.environment = "longtable",
  hline.after = FALSE,
  format.args = list(digits = 3, big.mark = ",")
)

In [ ]:
add.to.row <- list()
add.to.row$pos <- list()
add.to.row$pos[[1]] <- c(0)
add.to.row$command <- 
  paste0(
    "\\hline \n",
    "\\endfirsthead \n",
    "\\multicolumn{4}{p{\\textwidth}}{{ \\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\ \n",
    "\\hline Feature & No Preeclampsia/Eclampsia & Preeclampsia/Eclampsia & Overall \\\\ \\hline \n",
    "\\endhead \n",
    "\\hline \\multicolumn{4}{p{\\textwidth}}{{Continued on next page}} \\\\ \\hline \n",
    "\\endfoot \n",
    "\\hline \n",
    "\\endlastfoot \n"
  )

#align <- c('', "p{0.2\\textwidth}", "p{0.2\\textwidth}", "p{0.6\\textwidth}")

caption <- 
  paste0(
    'Demographic characteristics of people that delivered at or after January 1, 2018 and their comorbidities. '
  )

label <- 'tab:table_1'

xtable::print.xtable(
  xtable::xtable(
    tab1_df,
    label = label,
    caption = caption,
    floating = FALSE#,
    #align = align
  ),
  type = 'html', 
  file = file.path(results_dir, "table_1.html"), 
  include.rownames = FALSE,
  comment = FALSE,
  timestamp = NULL,
  floating = FALSE,
  add.to.row = add.to.row,
  tabular.environment = "longtable",
  hline.after = FALSE,
  format.args = list(digits = 3, big.mark = ",")
)

# Kaplan Meier Curves

In [ ]:
# Create plotting function
custom_theme <- \() {
  theme_truveta() %+replace%
    theme(
      plot.title=element_text(hjust=0.5),
      axis.title.x = element_text(hjust=0.5)
    )
}



## Combined Figure 2

In [ ]:
#survival plot for preeclampsia and HF all time
j <- survival::survfit(survival::Surv(time_hf, event_hf) ~ preeclampsia_eclampsia_pretty, data = df_surv)

#survival plot for preeclampsia and HF 90 days
k <- survival::survfit(survival::Surv(time90_hf, event_hf) ~ preeclampsia_eclampsia_pretty, data = df_surv)

choose <-  ggsurvplot(k,
    data=df_surv, 
    conf.int = TRUE, 
    color = "preeclampsia_eclampsia_pretty", 
    palette = official_color_palette,
    xlim = c(0, 90),
    break.x.by = 30,
    title = "Time to Heart Failure",
    ylab = "Proportion not Experiencing Heart Failure",
    xlab = "Time Since Delivery (days)",
    ylim = c(.98,1), ggtheme = theme_truveta()
  )

choose2 <- ggsurvplot(
    j,
    data=df_surv, 
    conf.int = TRUE, 
    color = "preeclampsia_eclampsia_pretty", 
    palette = official_color_palette,
      xscale=365.25,
                     break.time.by = 365.25,
    #xlim = c(0, 90),
    #break.x.by = 30,
    title = "Time to Heart Failure",
    ylab = "Proportion not Experiencing Heart Failure",
    xlab = "Time Since Delivery (years)",
    ylim = c(.98,1), ggtheme = theme_truveta()
  )


splots <-list(choose, choose2)

In [ ]:
a <- choose$plot
b <- choose2$plot

In [ ]:
a / b

In [ ]:
cheese <- ggplot2::labs(
 title = NULL,
 subtitle = 'B. Survival Without Heart Failure within 90 Days of Delivery',
 x = 'Time Since Delivery (days)',
 y = stringr::str_wrap('Proportion not Experiencing Heart Failure', 30))

a + cheese

In [ ]:
cheese2 <- ggplot2::labs(
 title = stringr::str_wrap('Time to Heart Failure After Delivery by Preeclampsia Status', 40),
 subtitle = 'A. Survival Without Heart Failure Occurrence Any Time After Delivery',
 x = 'Time Since Delivery (years)',
 y = stringr::str_wrap('Proportion not Experiencing Heart Failure', 30))

b + cheese2

In [ ]:
res <- (b + cheese2) / (a + cheese)

In [ ]:
ggsave(plot = res, device = "png", filename = file.path(results_dir, "KM_Dual_Plot_Overall_98.png"))

## Combined Figure 3 by Race

In [ ]:
# moving this block here because otherwise notebook won't run in order

#Re-level variables in advance of the multivariate in order to get references correct
df_surv2 <- 
  df_surv |>
  mutate(
    race = forcats::fct_relevel(race, c("White", "Black", "Asian", "Other Race")),
    Advanced = forcats::fct_relevel(Advanced, c("No","Yes")),
    Teen = forcats::fct_relevel(Teen, c("No","Yes")),
    ethnicity = forcats::fct_relevel(ethnicity, c("Not Hispanic or Latino","Hispanic or Latino"))
  )


In [ ]:
#survival plot for preeclampsia and HF all time
m <- survival::survfit(survival::Surv(time_hf, event_hf) ~ preeclampsia_eclampsia_pretty + race, data = df_surv2)

#survival plot for preeclampsia and HF 90 days
mm <- survival::survfit(survival::Surv(time90_hf, event_hf) ~ preeclampsia_eclampsia_pretty + race, data = df_surv2)



cheese3 <- ggplot2::labs(
  title = stringr::str_wrap('Time to Heart Failure at or After Delivery by Preeclampsia Status and Race', 40),
  subtitle = stringr::str_wrap('A. Survival Without Heart Failure Occurrence Any Time at or After Delivery', 40),
  x = 'Time Since Delivery (years)',
  y = stringr::str_wrap('Proportion not Experiencing Heart Failure', 30)
)


innie <- 
  ggsurvplot_facet(
    m, 
    df_surv2, 
    facet.by = "race",
    short.panel.labs=T ,  
    ncol=5,          
    conf.int = TRUE, 
    color = "preeclampsia_eclampsia_pretty",
    palette = official_color_palette,
    xscale=365.25,
    break.time.by = 365.25,
    title = "Time to Heart Failure by Race",
    ylab = "Proportion not Experiencing Heart Failure",
    xlab = "Time Since Delivery (years)",
    ylim = c(.98,1),ggtheme = theme_truveta()  
  )


innie + cheese3 + facet_wrap(~forcats::fct_relevel(race,"White", "Black", "Asian", "Other Race"),ncol=5)

In [ ]:
outtie <-  ggsurvplot_facet(
  mm,
  df_surv2, facet.by="race", 
  short.panel.labs=T, 
  ncol=5,
  conf.int = TRUE, 
  color = "preeclampsia_eclampsia_pretty", 
  palette = official_color_palette,
  xlim = c(0, 90),
  break.x.by = 30,
  title = "Time to Heart Failure by Race",
  ylab = "Proportion not Experiencing Heart Failure",
  xlab = "Time Since Delivery (days)",
  ylim = c(.98,1), ggtheme = theme_truveta()
)


outtie + cheese + facet_wrap(~forcats::fct_relevel(race,"White", "Black", "Asian", "Other Race"),ncol=5)

In [ ]:
reserve <- (innie + cheese3 + facet_wrap(~forcats::fct_relevel(race,"White", "Black", "Asian", "Other Race"),ncol=5)) / (outtie + cheese + facet_wrap(~forcats::fct_relevel(race,"White", "Black", "Asian", "Other Race"),ncol=5))

In [ ]:
reserve

In [ ]:
ggsave(plot = reserve, device = "png", filename = file.path(results_dir, "KM_Dual_Plot_By_Race_98.png"))

# Cox PH Models

In [ ]:
#Original model
# No interactions
original <- 
  survival::coxph(
    survival::Surv(time_hf, event_hf) ~ 
    preeclampsia_eclampsia_pretty + 
 race + 
      ethnicity + 
      AgeGroup + 
      bleeding_conditions +
      chronic_diabetes_conditions + 
      gest_diabetes_conditions + 
     CountHBP + 
      prexist_cardiac_exclusion +
     bmiabove40_conditions + 
      sud_conditions + 
      disabled2 + 
      oci, 
    data = df_surv2
  )


cox1 <- original |> broom::tidy(exp = TRUE, conf.int = TRUE) 
#export the results into a data frame to create the csv and tex files
#HIV, gestational diabetes, lupus, and bleeding disorders do not appear to influence the relationship. not significant. continue testing

In [ ]:
cox1

In [ ]:
#Whew this works! Makes the last line unneeded from previous cell.
#This turns the results into a gt table that can be saved as HTML, TEX, Word, etc.
tab1 <-  
  tbl_regression(original, exponentiate=TRUE) |> 
  as_gt()
#could also use |> as_latex()

## fit model with original names to output regression table

In [ ]:
library(lmtest)

In [ ]:
#Redone model selection. Full is all variables and potential interactions
full <- 
  survival::coxph(
    survival::Surv(time_hf, event_hf) ~ 
    preeclampsia_eclampsia_pretty + 
 race + 
      ethnicity + 
      AgeGroup + 
      bleeding_conditions +
      chronic_diabetes_conditions + 
      gest_diabetes_conditions + 
     CountHBP + 
      prexist_cardiac_exclusion +
     bmiabove40_conditions + 
      sud_conditions + 
      disabled2 + 
      oci +
      
      
        preeclampsia_eclampsia_pretty*race + 
    chronic_diabetes_conditions*race + 
    gest_diabetes_conditions*race + 
      CountHBP*race +
      bmiabove40_conditions*race + 
    sud_conditions*race + 
    disabled2*race + 

      preeclampsia_eclampsia_pretty*ethnicity + 
    preeclampsia_eclampsia_pretty*AgeGroup + 
      preeclampsia_eclampsia_pretty*CountHBP +
    preeclampsia_eclampsia_pretty*bmiabove40_conditions + 
    preeclampsia_eclampsia_pretty*sud_conditions + 
    
    preeclampsia_eclampsia_pretty*disabled2 + 
    CountHBP*chronic_diabetes_conditions       
      
      
      
      , 
    data = df_surv2
  )
summary(full)

In [ ]:
#reduced1 removes     preeclampsia_eclampsia_pretty*AgeGroup due to convergence issues.     CountHBP*chronic_diabetes_conditions   
reduced1 <- 
  survival::coxph(
    survival::Surv(time_hf, event_hf) ~ 
    preeclampsia_eclampsia_pretty + 
 race + 
      ethnicity + 
      AgeGroup + 
      chronic_diabetes_conditions + 
      gest_diabetes_conditions + 
     CountHBP + 
      prexist_cardiac_exclusion +
     bmiabove40_conditions + 
      sud_conditions + 
      disabled2 + 
      oci +

        preeclampsia_eclampsia_pretty*race + 
    sud_conditions*race + 
    disabled2*race + 

    preeclampsia_eclampsia_pretty*sud_conditions + 
    preeclampsia_eclampsia_pretty*disabled2

      ,
    data = df_surv2
  )
summary(reduced1)

In [ ]:
AIC(original, reduced1, full)
#lowest AIC is reduced1

In [ ]:
lrtest(original, reduced1, full)
#reduced1 is the best model

In [ ]:
if(!requireNamespace("kimisc")) {
  install.packages("kimisc")
}
if(!requireNamespace("AICcmodavg")) {
  install.packages("AICcmodavg")
}
library(AICcmodavg)
library(kimisc)

In [ ]:
model.set <- list(original, reduced1)
model.names <- c("original","reduced1")

aictab(model.set, test='LRT', modnames = model.names)

In [ ]:
Unadjusted <- original
Adjusted <- reduced1

In [ ]:
# if(!requireNamespace("kimisc")) {
#   install.packages("kimisc")
# }
# if(!requireNamespace("AICcmodavg")) {
#   install.packages("AICcmodavg")
# }

require(survival)
require(kimisc) # has the nlist function to create a named list
require(AICcmodavg) # has the aictab function
require(dplyr)
require(ggplot2)
require(reshape2)



# Put the models all together in a named list using nlist function from kimisc package
model_list <- nlist(Unadjusted, Adjusted)

# Compare models with AIC table
aic_table <- aictab(model_list, second.ord=FALSE)

aic_table_small = subset(aic_table, select = -c(K,ModelLik,LL,Cum.Wt,AICWt) )


# Add a column to the table with variable names
model_vars <- sapply(model_list, function(x) gsub(":", "*", paste(names(x$means), collapse = " + ")))
aic_table_small2 <- cbind(Modnames = names(model_vars)) %>%
  merge(aic_table_small, by="Modnames") %>%
  arrange(AIC)

# Add a column to the table with variable names
model_vars <- sapply(model_list, function(x) gsub(":", "*", paste(names(x$means), collapse = " + ")))
aic_table_full2 <- cbind(Modnames = names(model_vars), model_vars) %>%
  merge(aic_table, by="Modnames") %>%
  arrange(AIC)

In [ ]:
aic_table
aic_table_full2
aic_table_small
aic_table_small2

In [ ]:
library(xtable)

In [ ]:
print(xtable(aic_table_small2, type = "latex", tabular.environment="longtable"), file = file.path(results_dir, "aic_table_small.tex"))

write.csv(aic_table_small2, file = file.path(results_dir, 'aic_table_small.csv'))

In [ ]:
print(xtable(aic_table_full2, type = "latex", tabular.environment="longtable"), file = file.path(results_dir, "aic_table_full.tex"))

write.csv(aic_table_full2, file = file.path(results_dir, 'aic_table_full.csv'))

### emmeans work

In [ ]:
# varaibles need to be redefined to be computer friendly
# human friendly states do not play well with post processing
df_surv_temp <- 
  df_surv2 |>
  dplyr::mutate(
    preeclampsia_eclampsia = 
      dplyr::if_else(
        preeclampsia_eclampsia_pretty == 'Preeclampsia/Eclampsia', 
        'yes', 
        'no'
      ),
    race = as.character(race),
    race = dplyr::if_else(race == 'Other Race', 'Other', race),
    race = forcats::fct_infreq(race)
  )

In [ ]:
#reduced1 removes     preeclampsia_eclampsia_pretty*AgeGroup due to convergence issues
emmeansmodel <- 
  survival::coxph(
    survival::Surv(time_hf, event_hf) ~ 

    preeclampsia_eclampsia + 
    race + 
    ethnicity + 
    AgeGroup + 
    chronic_diabetes_conditions +   
    gest_diabetes_conditions + 
    CountHBP +
    prexist_cardiac_exclusion +
    bmiabove40_conditions + 
    sud_conditions + 
    disabled2 + 
    oci +
    preeclampsia_eclampsia*race + 
    sud_conditions*race + 
    disabled2*race + 
    preeclampsia_eclampsia*sud_conditions + 
    preeclampsia_eclampsia*disabled2

      ,
    data = df_surv_temp
  )
#summary(reduced1)

#come back and reorder the interactions for interpretation purposes

In [ ]:
non_nuisance <- c("preeclampsia_eclampsia", "race", "sud_conditions", "disabled2")

rg <- emmeans::ref_grid(emmeansmodel, non.nuisance = non_nuisance)

emms <- emmeans::emmeans(object = rg, specs = non_nuisance)

prs <- pairs(emms, reverse = FALSE)
prs_rev <- pairs(emms, reverse = TRUE)
# sometimes you want reverse = TRUE
# if you can't find the comparison you want in coming out of comparisons, change reverse to TRUE

comparisons <- confint(prs, type = 'response')
comparisons_rev <- confint(prs_rev, type = 'response')

# which comparisons do you want to interact with? there are 120
numer <- paste0('num_', non_nuisance)
denom <- paste0('denom_', non_nuisance)

make_readable <- function(comparisons, numerators, denominators) {
  readable <-
    comparisons |>
    tibble::as_tibble() |>
    tidyr::separate(contrast, into = c('num', 'denom'), sep = ' / ') |>
    tidyr::separate(num, into = numer, sep = ' ') |>
    tidyr::separate(denom, into = denom, sep = ' ') |>
    dplyr::rename(
      lower_ci = asymp.LCL,
      upper_ci = asymp.UCL
    )
  
  readable
}

comp_read <- make_readable(comparisons, numer, denom)
comp_rev_read <- make_readable(comparisons_rev, numer, denom)

compare <- dplyr::bind_rows(comp_read, comp_rev_read)

In [ ]:
# black + pre + nosud + nodisabled / black + nopre + nosud + nodisabled
# black + pre + nosud + disabled   / black + nopre + nosud + disabled
# black + pre + sud +   nodisabled / black + nopre + sud +   nodisabled
# black + pre + sud +   disabled   / black + nopre + sud +   disabled

black_blacknopre <- 
  compare |>
  dplyr::filter(
    num_race == 'Black',
    denom_race == 'Black',
    num_preeclampsia_eclampsia == 'yes',
    denom_preeclampsia_eclampsia == 'no',
    num_sud_conditions == denom_sud_conditions,
    num_disabled2 == denom_disabled2
  )



# black + pre + nosud + nodisabled / white + pre + nosud + nodisabled
# black + pre + nosud + disabled   / white + pre + nosud + disabled
# black + pre + sud +   nodisabled / white + pre + sud +   nodisabled
# black + pre + sud +   disabled   / white + pre + sud +   disabled

black_whitepre <- 
  compare |>
  dplyr::filter(
    num_race == 'Black',
    denom_race == 'White',    
    num_preeclampsia_eclampsia == 'yes',
    denom_preeclampsia_eclampsia == 'yes',
    num_sud_conditions == denom_sud_conditions,
    num_disabled2 == denom_disabled2
  )



# black + pre + nosud + nodisabled / white + nopre + nosud + nodisabled
# black + pre + nosud + disabled   / white + nopre + nosud + disabled
# black + pre + sud +   nodisabled / white + nopre + sud +   nodisabled
# black + pre + sud +   disabled   / white + nopre + sud +   disabled

black_whitenopre <- 
  compare |>
  dplyr::filter(
    num_race == 'Black',
    denom_race == 'White',    
    num_preeclampsia_eclampsia == 'yes',
    denom_preeclampsia_eclampsia == 'no',
    num_sud_conditions == denom_sud_conditions,
    num_disabled2 == denom_disabled2
  )



# black + nopre + nosud + nodisabled / white + nopre + nosud + nodisabled
# black + nopre + nosud + disabled   / white + nopre + nosud + disabled
# black + nopre + sud +   nodisabled / white + nopre + sud +   nodisabled
# black + nopre + sud +   disabled   / white + nopre + sud +   disabled

blacknopre_whitenopre <- 
  compare |>
  dplyr::filter(
    num_race == 'Black',
    denom_race == 'White',    
    num_preeclampsia_eclampsia == 'no',
    denom_preeclampsia_eclampsia == 'no',
    num_sud_conditions == denom_sud_conditions,
    num_disabled2 == denom_disabled2
  )



# white + pre + nosud + nodisabled / white + nopre + nosud + nodisabled
# white + pre + nosud + disabled   / white + nopre + nosud + disabled
# white + pre + sud +   nodisabled / white + nopre + sud +   nodisabled
# white + pre + sud +   disabled   / white + nopre + sud +   disabled

white_whitenopre <- 
  compare |>
  dplyr::filter(
    num_race == 'White',
    denom_race == 'White',    
    num_preeclampsia_eclampsia == 'yes',
    denom_preeclampsia_eclampsia == 'no',
    num_sud_conditions == denom_sud_conditions,
    num_disabled2 == denom_disabled2
  )


compare_focus <- 
  dplyr::bind_rows(
    black_blacknopre,
    black_whitepre,
    black_whitenopre,
    blacknopre_whitenopre,
    white_whitenopre
  ) |>
  dplyr::mutate(
    dplyr::across(c(num_preeclampsia_eclampsia, denom_preeclampsia_eclampsia), ~ dplyr::if_else(.x == 'yes', 'Yes', 'No'))
  ) |>
  dplyr::rename(hazard_ratio = ratio) |>
  dplyr::select(-SE, -df)

write.csv(compare_focus, file = file.path(results_dir, 'hazard_ratio_comparisons.csv'))

In [ ]:
library(gt)

In [ ]:
caption <- 
  paste0(
    'Hazard ratios associated comparisons between different combinations of preclampsia/eclampsia status (Yes/No), substance use disorder (Yes/No), physical disability (Yes/No) and their interaction with race (White/Black). The state of each of these variables is shown in the [N]umerator and [D]enominator columns beneath each variable name. Hazard ratio is provided with 95% confidence intervals.'
  )

compare_focus_gt <- 
  compare_focus |>
  arrange(desc(hazard_ratio)) |>
  gt::gt() |>
  fmt_number(
    columns = c(hazard_ratio, lower_ci, upper_ci)
  ) |>
  cols_merge(
    columns = c("lower_ci", "upper_ci"),
    pattern = "[{1}, {2}]"
  ) |>
  cols_label(
    num_preeclampsia_eclampsia = 'N',
    denom_preeclampsia_eclampsia = 'D',
    num_race = 'N',
    denom_race = 'D',
    num_sud_conditions = 'N',
    denom_sud_conditions = 'D',
    num_disabled2 = 'N',
    denom_disabled2 = 'D',
    hazard_ratio = 'Hazard Ratio',
    lower_ci = '[95%]'
  ) |>
  tab_spanner(
    label = "Preeclampsia/Eclampsia",
    columns = c("num_preeclampsia_eclampsia", "denom_preeclampsia_eclampsia")
  ) |>
  tab_spanner(
    label = "Race",
    columns = c("num_race", "denom_race")
  ) |>
  tab_spanner(
    label = "Substance Use Disorder",
    columns = c("num_sud_conditions", "denom_sud_conditions")
  ) |>
  tab_spanner(
    label = "Disabilities",
    columns = c("num_disabled2", "denom_disabled2")
  )


gtsave(compare_focus_gt, "hazard_ratio_table.html", path = results_dir)
gtsave(compare_focus_gt, "hazard_ratio_table.tex", path = results_dir)

In [ ]:
hr_matrix <- 
  compare |>
  dplyr::filter(
    num_race %in% c('Black', 'White'),
    denom_race %in% c('Black', 'White')
  ) |>
  dplyr::mutate(
    numerator = paste(num_preeclampsia_eclampsia, num_race, num_sud_conditions, num_disabled2),
    denominator = paste(denom_preeclampsia_eclampsia, denom_race, denom_sud_conditions, denom_disabled2),
    across(c(numerator, denominator), ~ stringr::str_to_title(.x)),
    across(c(numerator, denominator), ~ stringr::str_replace_all(.x, pattern = '[a-z]', replacement = '')),
    across(c(numerator, denominator), ~ stringr::str_replace_all(.x, pattern = ' ', replacement = ''))    
  ) |>
  dplyr::select(
    numerator, denominator, ratio#, lower_ci, upper_ci
  ) |>
  dplyr::arrange(desc(ratio)) |>
  tidyr::pivot_wider(
    names_from = denominator,
    values_from = ratio
  ) |> 
  rev() |>
  dplyr::relocate(numerator, .before = everything()) |>
  dplyr::mutate(dplyr::across(-numerator, ~ round(.x, 2))) |>
  dplyr::rename(`N/D` = numerator)

hr_matrix <- as.matrix(hr_matrix)
diag(hr_matrix[, -1]) <- 1

hr_matrix <- tibble::as_tibble(hr_matrix)

hr_matrix

caption <- 
  paste0('Matrix of hazard ratios comparing between patients by preeclampsia/eclampsia ([Y]es/[N]o), race ([B]lack/[W]hite),',
         ' substance use disorder ([Y]es/[N]o), and disability ([Y]es/[N]o). ',
         'Row names represent the numerator (N) of the hazard ratio, and columns names the denominator (D). ')

readr::write_csv(hr_matrix, file = file.path(results_dir, 'hazard_ratios_matrix.csv'))
write_as_xtable(hr_matrix, filepath = file.path(results_dir, 'hazard_ratios_matrix.tex'), caption = caption)

In [ ]:
library(gtsummary)
library(gt)

# Supplemental Table 2

In [ ]:
# clean interpretations of exp(coef) not possible with interaction terms
# also creates complciations about *what* is included in the hazard ratio
# exp(coef) = HR works because the HR is comparing risk where x = 1 / risk where x = 0
# for interactions, you sum the coefs assocaited with your comparison and then exp that
# this is complicated when you're trying to make specific population comparisons
# in general, a big table of regression coefficients from a model with interactions with be hard to quickly read
multitab <-  
  tbl_regression(
    reduced1, 
    exponentiate = TRUE,
    list(
      preeclampsia_eclampsia_pretty ~ "Preeclampsia/Eclampsia", 
      race ~ "Race", 
      ethnicity ~ "Ethnicity", 
      AgeGroup ~ "Age Group", 
      chronic_diabetes_conditions ~ "Pregestational Diabetes (Type 1 and Type 2)", 
        CountHBP ~ "Pregestational Hypertension",
      bmiabove40_conditions ~ "BMI U+2265 40", 
      disabled2 ~ "Disability", oci ~ "Obstetric Comorbidity Index Score",
gest_diabetes_conditions ~ 'Gestational Diabetes',
prexist_cardiac_exclusion ~ 'Other Preexisting Cardiac Conditions',
sud_conditions~'Substance Use Disorder'
)
      
      
  ) |>
  as_gt() |>
  gt::tab_header(title = gt::md("**Regression coefficients from a Cox proportional hazards model describing time till heart failure.**"))

In [ ]:
gtsave(multitab, "supptab_2.html", path = results_dir)
gtsave(multitab, "supptab_2.tex", path = results_dir)

In [ ]:
# clean interpretations of exp(coef) not possible with interaction terms
# also creates complciations about *what* is included in the hazard ratio
# exp(coef) = HR works because the HR is comparing risk where x = 1 / risk where x = 0
# for interactions, you sum the coefs assocaited with your comparison and then exp that
# this is complicated when you're trying to make specific population comparisons
# in general, a big table of regression coefficients from a model with interactions with be hard to quickly read
multitab2 <-  
  tbl_regression(
    reduced1, 
    exponentiate = FALSE, estimate_fun = purrr::partial(style_ratio, digits = 2),
    list(
      preeclampsia_eclampsia_pretty ~ "Preeclampsia/Eclampsia", 
      race ~ "Race", 
      ethnicity ~ "Ethnicity", 
      AgeGroup ~ "Age Group", 
      chronic_diabetes_conditions ~ "Pregestational Diabetes (Type 1 and Type 2)", 
        CountHBP ~ "Pregestational Hypertension",
      bmiabove40_conditions ~ "BMI U+2265 40", 
      disabled2 ~ "Disability", oci ~ "Obstetric Comorbidity Index Score",
gest_diabetes_conditions ~ 'Gestational Diabetes',
prexist_cardiac_exclusion ~ 'Other Preexisting Cardiac Conditions',
sud_conditions~'Substance Use Disorder'
)
      
      
  ) |>
  as_gt() |>
  gt::tab_header(title = gt::md("**Regression coefficients from a Cox proportional hazards model describing time till heart failure.**"))

In [ ]:
gtsave(multitab2, "supptab_3.html", path = results_dir)
gtsave(multitab2, "supptab_3.tex", path = results_dir)

# print the r packages that were used here

In [ ]:
(.packages())

In [ ]:
knitr::write_bib(file = file.path(results_dir, 'core_r.bib'))

In [ ]:
packs <- 
  c(
    "arrow",
    "dplyr",
    "ggplot2",
    "janitor",
    "broom",
    "emmeans",
    "scales",
    "survminer",
    "survival",
    "ggfortify",
    "gtools",
    "MASS",
    "xtable",
    "bit64",
    "table1",
    "patchwork",
    "DiagrammeR",
    "DiagrammeRsvg",
    "magrittr",
    "gt",
    "labelled",
    "table1",
    "ggpubr",
    "stats",
    "grDevices"
  )

# knitr::write_bib to export bib file with all the references. saves you copy-pasting
knitr::write_bib(packs, file = file.path(results_dir, 'packages.bib'))

# Study Flowchart / MUST BE RUN IN DESKTOP VERSION OF R OR RSTUDIO. WILL NOT RUN HERE!

In [ ]:
rsvg <- "https://cran.r-project.org/src/contrib/Archive/rsvg/rsvg_2.2.0.tar.gz"
install.packages(rsvg, repos=NULL, type="source")

In [ ]:
install.packages(c("DiagrammeR","DiagrammeRsvg","magrittr","librsvg"))
library(DiagrammeR)
library(DiagrammeRsvg)
library(magrittr)
library(librsvg)
library(rsvg)

In [ ]:
flowy1 <- grViz("digraph flowchart {
      # node definitions with substituted label text
      node [fontname = Helvetica, shape = rectangle]        
      tab1 [label = '@@1']
      tab2 [label = '@@2']
      tab3 [label = '@@3']
      tab4 [label = '@@4']
      tab5 [label = '@@5']
        tab6 [label = '@@6']
tab7 [label = '@@7']
tab8 [label = '@@8']
#the boxes to the side (exclude level) - I need just one (excluded) off of box 2
    node [fontname = Helvetica, shape = rectangle]
      m1 [label = 'Did not meet study criteria n=678,792']


#draw arrows to the different boxes for the first groups before final groupings
# creating horizontal lines
      node [shape=none, width=0, height=0, label='']

      {rank=same; tab2 -> m1}
     
      # edge definitions with the node IDs
      tab1 -> tab2;
      tab2 -> tab3 -> {tab7 tab8}
      tab2 -> tab4 -> {tab5 tab6};
            }

      [1]: 'Patients with a delivery at or after January 1, 2018\\n n=1,376,218'
      [2]: 'Patients eligible for the study\\n n=697,426'
      [3]: 'Eligible patients with pre-eclampsia or eclampsia\\n n=41,982'
      [4]: 'Eligible patients without pre-eclampsia or eclampsia\\n n=655,444'
      [5]: 'Patients with any heart failure\\n n=1478'
      [6]: 'Patients without any heart failure\\n n=653,966'
      [7]: 'Patients with any heart failure\\n n=399'
      [8]: 'Patients without any heart failure\\n n=41,583'
      ")

In [ ]:
flowy1

In [ ]:
#mypath <- file.path("C:","R","SAVEHERE",paste("myplot_", names[i], ".jpg", sep = ""))

results_dir <- here::here("results")

In [ ]:
images_dir <- file.path(results_dir, "images")
dir.create(images_dir, recursive = TRUE, showWarnings = FALSE)

In [ ]:
library(V8)

In [ ]:
rsvg <- "https://cran.r-project.org/src/contrib/Archive/rsvg/rsvg_2.2.0.tar.gz"

install.packages(rsvg, repos=NULL, type="source")
install.packages(c("DiagrammeR","DiagrammeRsvg","magrittr","xml2"))

library(DiagrammeR)
library(DiagrammeRsvg)
library(magrittr)
library(rsvg)
library(xml2)


# flowchart ---------------------------------------------------------------


#call the object whatever. i used flowy1
par(family = "roman")
flowy1 <- DiagrammeR::grViz("digraph flowchart {

#Give the study chart a title

#Where should the title go?
labelloc = b
#What font size should the title have?
fontsize = 14
#What font should the title be?
fontname = 'roman'



#what default font and shape do i want the nodes to be?
node [fontname = 'roman', shape = rectangle, fontsize = 12]



# node definitions with substituted label text. without labels,
#the node would just be whatever you list (ex: total).
#call the nodes what you want. 
#here they are named based on what they represent to make
#ordering them easier. 


#these go downward in the flowchart


#The first node is called 'total'. The label statement allows HTML use.
#I used it here to make the top line bold in the nodes. I then refer to 
#a shortcut I made below to more easily find and edit the numbers '@@1'


total      [label = <
Total patients assessed for study eligibility<br/>
@@1
>, width='5']
       
       
#Even though the boxes are by default resized for text, the use of HTML here 
#causes some issues so I resized each box accordingly using the width statement

included   [label = <
Individual patients included in the study<br/>
@@2
>, width='4']



pree       [label = <
Preeclampsia or eclampsia<br/>
@@3
>, width='3']



prehf      [label = <
Heart failure<br/>
@@4
>, width='1.5']



prenohf    [label = <
No heart failure<br/>
@@5
>, width='1.5']
       
       
       
       
no   [label = <
No preeclampsia or eclampsia<br/>
@@6
>, width='3']




hfnopree   [label =  <
Heart failure<br/>
@@7
>, width='1.5']




doublenohf  [label = <
No heart failure<br/>
@@8
>, width='1.5']




#these next nodes go horizontally
#remember you can actually call the nodes whatever you want.

exclude1    [label = <
Total patients excluded: @@9<br/><br/>
Patients were excluded for:<br/>
last clinical encounter on same day<br/> or before delivery indicator <br/>or unrealistic future date; <br/>
having heart failure, cardiomyopathy,<br/> chronic kidney disease,<br/>
myocardial infarction, pulmonary<br/> embolism, or other cardiac<br/>
conditions within 365 days prior to <br/>delivery indicator;<br/>
having congenital heart disease;<br/>
missing or unknown race;<br/>
missing or unknown ethnicity;<br/> or less than 16 years of age or more <br/>than 50 years of age at<br/>
time of delivery indicator.<br/>
>, width='3']





      
#the next ones are blank nodes used to space out the boxes/other nodes
#while they are nodes like the others, they won't physically appear 
#because of my settings to make sure they are so small you just see a 
#continued arrow instead without these, the arrows and the boxes 
#aren't as clear

      blank1 [shape=none, width=0.01, height=0.01, label='']
      blank2 [shape=none, width=0.01, height=0.01, label='']
      blank3 [shape=none, width=0.01, height=0.01, label='']
      blank4 [shape=none, width=0.01, height=0.01, label='']



# whereas nodes are the individual things you are connecting, edges are
# how you connect them. here we have edge definitions with the node IDs

# currently the default is normal arrows.
# find other options: https://graphviz.org/doc/info/arrows.html


# are there any boxes that should be on the same level or side 
# by side without an adjoining arrow? this is a cool way to get sharp lines!

{rank= same; blank1 exclude1};
{rank= same; blank2 pree};
{rank= same; blank2 no}
{rank= same; blank3 blank4}



# how should the boxes be laid out and what should connect them?

# patients assessed to a blank box. the dir = none means that the arrow has no head
total -> blank1 [dir=none];

# go from that first blank box to the exclusion criteria boxes. minlen sets the
# minimum length of the arrow

blank1 -> exclude1[minlen=2];



# go from the first blank box to the box with the total included in it
blank1 -> included[minlen=1];

# go from the total included box to the second blank box
included -> blank2[dir=none];


# use the second blank box to draw a straight horizontal line and connect
# that line to the two subcategories of patients. those subcategories go to
# the sub-sub categories. dir = back makes the arrow go right to left. the
# box that should go on the end of the arrow (with the arrowhead), should
# go first!

#subcategory

pree -> blank2[dir=back, minlen=3];
blank2 -> no[minlen=3] ;


# sub-sub-category 1. the brackets around prehf and prenohf here put them as
#subcategories on the same level

pree -> blank3[minlen=1, dir=none];
blank3 -> {prehf prenohf};


# sub-sub-category 2

no -> blank4[minlen=1,dir=none];
blank4 -> {hfnopree doublenohf};



   }


# at this point i'm referencing the tabs using the shortcuts from above
# technically i could write all of the text found in the label boxes above here, but additional
# formatting is more complicated here.

      [1]:  'n=1,021,659'
      [2]:  'n=718,165'
      [3]:  'n=14,204'
      [4]:  'n=69'
      [5]:  'n=14,135'      
      [6]:  'n=703,961'
      [7]:  'n=1,115'
      [8]:  'n=702,847'
      [9]:  'n=303,493'
       ")

#see what you created!
flowy1




# require(magrittr)
# require(DiagrammeR)
# require(DiagrammeRsvg)
# require(xml2)


DPI = 300
HeightCM = 50.8
WidthCM = 57.2431333
flowy1 %>% export_svg() %>% charToRaw %>% rsvg(width = WidthCM * (DPI/2.54), height = HeightCM * (DPI/2.54)) %>% png::writePNG("flowchart.png")
